# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_df = pd.read_csv("weather_data.csv")
weather_df.head()


,city,date,lat,temp,humd,wind,cloud
0,Qaanaaq,1613026590,77.4840,5.88,76,13.76,6
1,Mataura,1613026591,-46.1927,51.82,68,10.31,74
2,Kavaratti,1613026592,10.5669,82.40,53,6.78,35
3,Hobart,1613026593,-42.8794,75.00,40,12.66,0
4,Hithadhoo,1613026595,-0.6000,82.02,80,11.52,48


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
humidity = weather_df["humd"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["lat", "lng"]]

KeyError: "['lng'] not in index"

In [ ]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
fit_weather_df = weather_df.loc[(weather_df["temp"] > 70) & (weather_df["temp"] < 80) & (weather_df["cloud"] == 0), :]
fit_weather_df = fit_weather_df.dropna(how='any')
fit_weather_df.reset_index(inplace=True)
del fit_weather_df['index']
fit_weather_df.head()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
